<p style="font-family: Arial; font-size:1.75em;color:#003399; font-style:bold"><br>
 Ustawienia wyświetlania 
</p><br>
W tej części pokazane będzie w jaki sposób można wyświetlać obrazy czarno-białe na przykładzie danych z radaru obrazującego (SAR, Sentinel-1, ESA). Porównane są różne metody dopasowywania  kontrastu, aby wydobyć właściwą informację. W końcowej części pokazany jest analogiczny przykład dla obrazu satelitarnego z MSI (The Multispectral Instrument, Sentinel-2, ESA), zapisanego w formie jedynie 3 kanałów ( R = Band_4 [664 nm], G = Band_3 [560 nm], B = Band_2 [492 nm]).

<i>Obraz czarno-biały, 1 warstwa</i>

In [2]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import skimage 

In [4]:
from skimage import io
from skimage import transform

# Otwórz plik z dnia 6.02.2018 5:04:15, Sentinel-1, ESA
im = io.imread('S1B_EW_GRDM_1SDH_20180206T050415_Botnicka.tif')
plt.imshow(im,cmap=plt.get_cmap('gray'))
plt.show()

<i>---------Liniowe dopasowanie kontrastu--------</i>

In [6]:
from skimage import exposure, transform

# Zmniejsz rozdzielczość 10-krotnie
image = transform.downscale_local_mean(im,(10,10))

# liniowe dopasowanie kontrastu
minimum = np.min(image)
maximum = np.max(image)
upper = np.percentile(image,98)
lower = np.percentile(image,5)

res_minmax = exposure.rescale_intensity(image,in_range=(minimum, maximum)) # zakres min-max
res_perc = exposure.rescale_intensity(image,in_range=(lower, upper)) # zakres dolny - górlny percentyl
res_eq = exposure.equalize_hist(image) # wyrównanie histogramu

# Wyświetl Zatokę 
fig = plt.figure(figsize=(15, 10))
ax0 = plt.subplot(331)
ax0.imshow(res_minmax,cmap=plt.get_cmap('gray'))
ax0.set_title("Rescale minimum-maximum")
ax1 = plt.subplot(332)
ax2 = plt.subplot(332)
ax1.imshow(res_perc,cmap=plt.get_cmap('gray'))
ax1.set_title("Rescaled p5 - p98")
ax2 = plt.subplot(333)
ax2.imshow(res_eq,cmap=plt.get_cmap('gray'))
ax2.set_title("Equalize histogram")

# porównaj przypisanie kolorów
ax3 = plt.subplot(334)
ax3.hist(image.ravel(), bins=100,  color='black')
ax4 = ax3.twinx()
ax4.scatter(image,res_minmax,color='red',s=2)
ax3.set_title("Stretch minimum-maximum")

ax5 = plt.subplot(335)
ax5.hist(image.ravel(), bins=100,  color='black')
ax6 = ax5.twinx()
ax6.scatter(image,res_perc,color='red',s=2)
ax5.set_title("Stretch p5 - p98")

ax7 = plt.subplot(336)
ax7.hist(image.ravel(), bins=100,  color='black')
ax8 = ax7.twinx()
ax8.scatter(image,res_eq,color='red',s=2)
ax7.set_title("Stretch equalized")

# porównaj przypisanie kolorów w powiększeniu
ax9 = plt.subplot(337)
ax9.hist(image.ravel(), bins=100,  color='black')
ax10 = ax9.twinx()
ax10.scatter(image,res_minmax,color='red',s=2)
ax9.set_xlim([0,1500])
ax9.set_title("Stretch minimum-maximum")

ax11 = plt.subplot(338)
ax11.hist(image.ravel(), bins=100,  color='black')
ax12 = ax11.twinx()
ax12.scatter(image,res_perc,color='red',s=2)
ax11.set_xlim([0,1500])
ax11.set_title("Stretch p5 - p98")

ax13 = plt.subplot(339)
ax13.hist(image.ravel(), bins=100,  color='black')
ax14 = ax13.twinx()
ax14.scatter(image,res_eq,color='red',s=2)
ax13.set_xlim([0,1500])
ax13.set_title("Stretch equalized")

fig.tight_layout()
plt.show()

<i>---------Nieliniowe dopasowanie kontrastu--------</i>

In [8]:
from skimage import exposure

# nieliniowe dopasowanie kontrastu
res_g05 = exposure.adjust_gamma(image,0.5) # dopasowanie gamma, dla współczynnika 0.5
res_g2 = exposure.adjust_gamma(image,2) # dopasowanie gamma, dla współczynnika 2
res_log = exposure.adjust_log(image,1) # dopasowanie logarytmiczne

# Wyświetl Zatokę 
fig = plt.figure(figsize=(15, 10))
ax0 = plt.subplot(331)
ax0.imshow(res_g05,cmap=plt.get_cmap('gray'))
ax0.set_title("Rescale gamma 0.5")
ax1 = plt.subplot(332)
ax2 = plt.subplot(332)
ax1.imshow(res_g2,cmap=plt.get_cmap('gray'))
ax1.set_title("Rescaled gamma 2")
ax2 = plt.subplot(333)
ax2.imshow(res_log,cmap=plt.get_cmap('gray'))
ax2.set_title("Rescale log")

# porównaj przypisanie kolorów
ax3 = plt.subplot(334)
ax3.hist(image.ravel(), bins=100,  color='black')
ax4 = ax3.twinx()
ax4.scatter(image,res_g05,color='red',s=2)
ax3.set_title("Stretch gamma 0.5")

ax5 = plt.subplot(335)
ax5.hist(image.ravel(), bins=100,  color='black')
ax6 = ax5.twinx()
ax6.scatter(image,res_g2,color='red',s=2)
ax5.set_title("Stretch gamma 2")

ax7 = plt.subplot(336)
ax7.hist(image.ravel(), bins=100,  color='black')
ax8 = ax7.twinx()
ax8.scatter(image,res_log,color='red',s=2)
ax7.set_title("Logarithmic stretch")

# porównaj przypisanie kolorów w powiększeniu
ax9 = plt.subplot(337)
ax9.hist(image.ravel(), bins=100,  color='black')
ax10 = ax9.twinx()
ax10.scatter(image,res_g05,color='red',s=2)
ax9.set_xlim([0,1500])
ax9.set_title("Stretch gamma 0.5")

ax11 = plt.subplot(338)
ax11.hist(image.ravel(), bins=100,  color='black')
ax12 = ax11.twinx()
ax12.scatter(image,res_g2,color='red',s=2)
ax11.set_xlim([0,1500])
ax11.set_title("Stretch gamma 2")

ax13 = plt.subplot(339)
ax13.hist(image.ravel(), bins=100,  color='black')
ax14 = ax13.twinx()
ax14.scatter(image,res_log,color='red',s=2)
ax13.set_xlim([0,1500])
ax13.set_title("Logarithmic stretch")

fig.tight_layout()
plt.show()

<i>---------Podstawowe transformacje obrazu--------</i>

In [9]:
from skimage import exposure
from matplotlib import colors

# dopasowanie kontrastu
original = exposure.equalize_hist(im)
small = exposure.equalize_hist(image)

f, axes = plt.subplots(1,2,figsize=(15, 8))
axes[0].imshow(original,cmap=plt.get_cmap('gray'))

# Zaznaczenie fragmentu, z którym będziemy pracować dalej
cm = colors.ListedColormap(["#FFFFFF00", '#CC0000'])
aoi = original * 0
aoi[1000:2000,1000:2000] = 1
axes[0].imshow(aoi,cmap=cm)

axes[0].set_title("Wczytany obraz")
axes[1].imshow(small,cmap=plt.get_cmap('gray'))
axes[1].set_title("10-krotnie zmniejszona rozdzielczość przestrzenna")
f.tight_layout()
plt.show()

In [10]:
from skimage import transform

# Obrót
ice = exposure.equalize_hist(im[1000:2000,1000:2000])
rot45 = transform.rotate(ice,45)   # obrót o 45 stopni w prawo
rot_45 = transform.rotate(ice,-45) # obrót o 45 stopni w lewo

f, axes = plt.subplots(1,3,figsize=(10, 8))
axes[0].imshow(ice,cmap=plt.get_cmap('gray'))
axes[0].set_title("Północna część Zatoki")
axes[1].imshow(rot45,cmap=plt.get_cmap('gray'))
axes[1].set_title("Obrót 45")
axes[2].imshow(rot_45,cmap=plt.get_cmap('gray'))
axes[2].set_title("Obrót -45")
plt.show()

In [11]:
from skimage.transform import warp, SimilarityTransform

# Przesunięcie obrazu
tform = SimilarityTransform(translation=(100, -50))
warpedF = warp(ice, tform)
warpedI = warp(ice, tform.inverse)

f, axes = plt.subplots(1,2,figsize=(10, 8))
axes[0].imshow(warpedF,cmap=plt.get_cmap('gray'))
axes[0].set_title("Przesunięcie o 50 px w dół i 100 w prawo")
axes[1].imshow(warpedI,cmap=plt.get_cmap('gray'))
axes[1].set_title("Przesunięcie przeciwne")
plt.show()

In [12]:
from skimage.transform import swirl

# Nałożenie zniekształcenia
swirled = swirl(ice,center=[500,500],strength=5,radius=800)
plt.imshow(swirled,cmap=plt.get_cmap('gray'))
plt.show()

In [13]:
from skimage import util

# Zmiana wartości na przeciwne (negatyw)
ice_inv = util.invert(ice)
plt.imshow(ice_inv,cmap=plt.get_cmap('gray'))
plt.show()

<i>Obraz wielokanałowy, tutaj złożony z 3 warstw odpowiadających czerwieni, zieleni i niebieskiemu<i/>

In [39]:
from skimage import io, img_as_float, img_as_ubyte

rgb = io.imread('S2A_MSIL1C_20170418T095031_subset_RGB.tif')
# Wypisz wymiary rastra
#print("Obraz składa się z %d warstw" % rgb.shape[2])

red = rgb[:,:,2]
green = rgb[:,:,1]
blue = rgb[:,:,0]

In [17]:
# Wyświetl Zatokę 
fig = plt.figure(figsize=(10, 3))

# Wygeneruj histogramy
ax1 = plt.subplot(131)
ax1.hist(red.ravel(), bins=100,  color='red')
ax1.set_title("Red band")

ax2 = plt.subplot(132)
ax2.hist(green.ravel(), bins=100,  color='green')
ax2.set_title("Green band")

ax3 = plt.subplot(133)
ax3.hist(blue.ravel(), bins=100,  color='blue')
ax3.set_title("Blue band")

fig.tight_layout()
plt.show()

# Wypisz zakresy wartości dla każdego z kanałów
#print("Red range: %d - %d\nGreen range: %d - %d\nBlue range: %d - %d" % 
#      (np.min(red),np.max(red),np.min(green),np.max(green),np.min(blue),np.max(blue)))

<i>---------Porównanie wybranych metod dopasowania kontrastu--------</i>

In [41]:
from skimage import exposure

def contrast_perc(band, d, u):
    lower = np.percentile(band,d)
    upper = np.percentile(band,u)
    rescaled = exposure.rescale_intensity(band,in_range=(lower, upper))
    
    return(rescaled)

red = transform.downscale_local_mean(red,(10,10))
green = transform.downscale_local_mean(green,(10,10))
blue = transform.downscale_local_mean(blue,(10,10))

# dopasowanie liniowe w zakresie percentyli 1 i 90
pD = 1; pU = 90
perc_red = contrast_perc(red, pD, pU)
perc_green = contrast_perc(green, pD, pU)
perc_blue = contrast_perc(blue, pD, pU)
perc = img_as_float(np.stack([perc_red, perc_green, perc_blue], axis=2))

# dopasowanie logarytmiczne
l = 16
log_red = exposure.adjust_log(red,l)
log_green = exposure.adjust_log(green, l)
log_blue = exposure.adjust_log(blue, l)

#Uwaga na BUG: https://github.com/matplotlib/matplotlib/issues/9391/
#przeskalowanie kanałów poniżej
log = img_as_float(np.stack([log_red/np.max(log_red), log_green/np.max(log_green), log_blue/np.max(log_blue)], axis=2))

# wyrównaj histogramy / equalize
eq_red = exposure.equalize_hist(red)
eq_green = exposure.equalize_hist(green)
eq_blue = exposure.equalize_hist(blue)
eq = img_as_float(np.stack([eq_red, eq_green, eq_blue], axis=2))

# Wyświetl Zatokę 
fig = plt.figure(figsize=(15, 10))
ax0 = plt.subplot(231)
ax0.imshow(perc,cmap=plt.get_cmap('gray'))
ax0.set_title("Percentile stretch")
ax1 = plt.subplot(232)
ax1.imshow(log,cmap=plt.get_cmap('gray'))
ax1.set_title("Log stretch")
ax2 = plt.subplot(233)
ax2.imshow(eq,cmap=plt.get_cmap('gray'))
ax2.set_title("Eqalize histogram")

# porównaj przypisanie kolorów w powiększeniu
ax3 = plt.subplot(234)
ax3.hist(red.ravel(), bins=100,  color='red')
ax4 = ax3.twinx()
ax4.scatter(red,perc_red,color='red',s=2)
ax3.set_xlim([0,5000])
ax3.set_title("Percentile stretch")

ax5 = plt.subplot(235)
ax5.hist(green.ravel(), bins=100,  color='green')
ax6 = ax5.twinx()
ax6.scatter(green,log_green,color='red',s=2)
ax5.set_xlim([0,5000])
ax5.set_title("Log stretch")

ax7 = plt.subplot(236)
ax7.hist(blue.ravel(), bins=100,  color='blue')
ax8 = ax7.twinx()
ax8.scatter(blue,eq_blue,color='red',s=2)
ax7.set_xlim([0,5000])
ax7.set_title("Eqalize histogram")

fig.tight_layout()
plt.show()

In [27]:
import os
import sys
import skimage
from skimage import io

collection = io.imread_collection("*.jpg")
print(collection)
print(type(collection))
io.imshow_collection(collection)